## Get a pathway file based on V14 but using the same filtering as scGWAS original authors

The filtering described by the original authors is below with our annotations of how we changed anything, if at all:
* "We collected the gene–gene relationship data from PathwayCommons (v12, data access date: 12/12/2019) to construct the background network. The data downloaded originally had 1,851,006 interaction pairs that were curated and integrated from the public pathway and interaction databases."
    * We downloaded this data from v14 (data access data: 09/04/2024) and it originally had 2,484,222 interaction pairs.
* "We excluded those interactions that were annotated as in-complex-with, because those genes tended to be co-expressed and might inflate the results."
    * We followed the same methodology but also ended up removing any interactions based on non-gene interactions as well.
* "We further excluded 2291 ribosomal genes and housekeeping genes defined by the HSIAO_HOUSEKEEPING_GENES set from MSigDB."
    * We removed the housekeeping genes using the same file they used (from their github)
    * Although unable to get the same ribosomal genes they used, we curated a list of 639 ribosomal genes according to HGNC and hand annotation. We also removed 64 Polymerase genes, totaling to removing 1,112 genes.
    * We also removed the following genes from the final list given they can sometimes be considered housekeeping genes: HSPA1B (Heat shock protein 70 family member 1B), PSMD3 (Proteasome 26S subunit, non-ATPase 3), ACTRT2 (Actin-related protein T2), TST (Thiosulfate sulfurtransferase), HSPA1L (Heat shock protein family A member 1 like), PABPN1L (Poly(A) binding protein nuclear 1 like)

* "In addition, for each gene–gene pair, we examined their genomic locations and excluded those pairs that were located within 50 kb of each other... Furthermore, we excluded all pairs whose interacting genes are located in the MHC region (chr6:26000000_34000000, hg19) due to the complex LD in this region.""
    * We did the same except that we used the hg38 MHC region

We ended 1,124,417 interaction pairs compared to their 805,375. This led to 560 genes being added that were not considered previously, none that clearly show housekeeping or ribosomal annotations.

In [1]:
library(data.table)

In [2]:
pc_hgnc <- fread("~/Downloads/pc-hgnc.txt.gz", fill=TRUE)#, fill=TRUE)
pc_hgnc[1:2,]
# downloaded from https://download.baderlab.org/PathwayCommons/PC2/v14/

PARTICIPANT_A,INTERACTION_TYPE,PARTICIPANT_B,INTERACTION_DATA_SOURCE,INTERACTION_PUBMED_ID,PATHWAY_NAMES,MEDIATOR_IDS
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
A1BG,controls-expression-of,A2M,pid,12456685;7678052;9794795,IL6-mediated signaling events,pid:pid_74043;pid:pid_74038
A1BG,interacts-with,ABCC6,BioGRID,21988832,,biogrid:MolecularInteraction_bebab183-9762-4322-b91f-e9a69a0c9e97___null__162374_


In [3]:
# split according to metadata (the file has the interactions AND metadata on each participant)

pc_hgnc[2484223,]
metadata = pc_hgnc[2484224:2524907,1:5]

metadata[1:2,1:5]
colnames(metadata) = c("PARTICIPANT", "PARTICIPANT_TYPE", "PARTICIPANT_NAME", "UNIFICATION_XREF", "RELATIONSHIP_XREF")
unique(metadata$PARTICIPANT_TYPE)
pc_hgnc = pc_hgnc[1:2484222,]

PARTICIPANT_A,INTERACTION_TYPE,PARTICIPANT_B,INTERACTION_DATA_SOURCE,INTERACTION_PUBMED_ID,PATHWAY_NAMES,MEDIATOR_IDS
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
PARTICIPANT,PARTICIPANT_TYPE,PARTICIPANT_NAME,UNIFICATION_XREF,RELATIONSHIP_XREF,,


PARTICIPANT_A,INTERACTION_TYPE,PARTICIPANT_B,INTERACTION_DATA_SOURCE,INTERACTION_PUBMED_ID
<chr>,<chr>,<chr>,<chr>,<chr>
chebi:31403,SmallMoleculeReference,cinnarizine,chebi:CHEBI:31403,
SNORD121B,RnaReference,SNORD121B mRNA,,


[1] "SmallMoleculeReference"                                                          
 [2] "RnaReference"                                                                    
 [3] "DnaRegionReference;ProteinReference;RnaReference"                                
 [4] "ProteinReference;RnaReference;RnaRegionReference"                                
 [5] "ProteinReference;RnaReference"                                                   
 [6] "DnaRegionReference;ProteinReference;RnaReference;RnaRegionReference"             
 [7] "DnaReference;ProteinReference;RnaReference;RnaRegionReference"                   
 [8] "DnaReference;ProteinReference;RnaReference"                                      
 [9] "DnaReference;DnaRegionReference;ProteinReference;RnaReference"                   
[10] "RnaReference;RnaRegionReference"                                                 
[11] "ProteinReference"                                                                
[12] "DnaRegionReference;RnaReference"                                                 
[13] "ProteinReference;RnaRegionReference"                                             
[14] "RnaRegionReference"                                                              
[15] "DnaReference;DnaRegionReference;ProteinReference;RnaReference;RnaRegionReference"
[16] "DnaReference;DnaRegionReference;ProteinReference"                                
[17] "DnaRegionReference"                                                              
[18] "DnaRegionReference;ProteinReference"                                             
[19] "DnaReference;ProteinReference"                                                   
[20] "DnaReference;RnaReference;RnaRegionReference"                                    
[21] "DnaRegionReference;ProteinReference;RnaRegionReference"                          
[22] "DnaReference;RnaReference"                                                       
[23] "DnaRegionReference;RnaReference;RnaRegionReference"                              
[24] "DnaReference;ProteinReference;RnaRegionReference"                                
[25] "DnaReference"

## 1. Remove those in complex with 

In [4]:
sort(unique(pc_hgnc$INTERACTION_TYPE))
old = nrow(pc_hgnc)
old
# remove the in-complex-with
pc_hgnc <- pc_hgnc[pc_hgnc$INTERACTION_TYPE != "in-complex-with",]
nrow(pc_hgnc)
cat("\n# Pairs lost due to in-complex-with:", old-nrow(pc_hgnc))

[1] ""                               "catalysis-precedes"            
 [3] "chemical-affects"               "consumption-controlled-by"     
 [5] "controls-expression-of"         "controls-phosphorylation-of"   
 [7] "controls-production-of"         "controls-state-change-of"      
 [9] "controls-transport-of"          "controls-transport-of-chemical"
[11] "in-complex-with"                "interacts-with"                
[13] "reacts-with"                    "used-to-produce"

[1] 2484222

[1] 2279905


# Pairs lost due to in-complex-with: 204317

## 1. Remove housekeeping genes
- Use the HSIAO_HOUSEKEEPING_GENES_new.txt found on their github 

In [5]:

# elcude houseeping genes
old = nrow(pc_hgnc)
housekeep <- fread("~/Downloads/HSIAO_HOUSEKEEPING_GENES_new.txt", 
                  header=FALSE)
housekeep = housekeep$V1
cat("\n# Housekeeping Genes:", length(housekeep))
pc_hgnc <- pc_hgnc[!pc_hgnc$PARTICIPANT_A %in% housekeep,]
nrow(pc_hgnc)
pc_hgnc <- pc_hgnc[!pc_hgnc$PARTICIPANT_B %in% housekeep,]
nrow(pc_hgnc)
cat("\n# Pairs lost due to housekeeping:", old-nrow(pc_hgnc))


# Housekeeping Genes: 405

[1] 2179584

[1] 2053927


# Pairs lost due to housekeeping: 225978

## 2. Remove ribosomal proteins and RNAs

In [6]:
old = nrow(pc_hgnc)
rib_prots <- fread("~/Downloads/ribosomal_proteins_HGNC_9.4.24.csv", skip=1, header=TRUE)
rib_prots[1,]
rib_prots = rib_prots[["Approved symbol"]]
rib_RNAs <- fread("~/Downloads/ribosomal_RNAs_HGNC_9.4.24.csv", skip=1, header=TRUE)
rib_RNAs[1,]
rib_RNAs = rib_RNAs[["Approved symbol"]]
length(rib_RNAs)
length(rib_prots)
rib_genes = union(rib_RNAs, rib_prots)
length(rib_genes)

pc_hgnc <- pc_hgnc[!pc_hgnc$PARTICIPANT_A %in% rib_genes,]
pc_hgnc <- pc_hgnc[!pc_hgnc$PARTICIPANT_B %in% rib_genes,]
nrow(pc_hgnc)

# hand annotated the remaining ribosomal genes not captured in these files from HGNC
rem_rib_genes = unique(pc_hgnc$PARTICIPANT_A[grep("^RPS|^RPL|^RPP", pc_hgnc$PARTICIPANT_A)])
rem_rib_genes = union(rem_rib_genes, 
                      unique(pc_hgnc$PARTICIPANT_B[grep("^RPS|^RPL|^RPP", pc_hgnc$PARTICIPANT_B)]))
#rem_rib_genes
rib_genes = union(rib_genes, rem_rib_genes)
length(rib_genes)
length(rem_rib_genes)
pc_hgnc <- pc_hgnc[!pc_hgnc$PARTICIPANT_A %in% rem_rib_genes,]
pc_hgnc <- pc_hgnc[!pc_hgnc$PARTICIPANT_B %in% rem_rib_genes,]
nrow(pc_hgnc)
cat("\n# Pairs lost due to ribosomal:", old-nrow(pc_hgnc))


HGNC ID (gene),Approved symbol,Approved name,Previous symbols,Aliases,Chromosome,Group
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
HGNC:14275,MRPL1,mitochondrial ribosomal protein L1,,"BM022,uL1m",4q21.1,Large subunit mitochondrial ribosomal proteins


HGNC ID (gene),Approved symbol,Approved name,Previous symbols,Aliases,Chromosome,Group
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
HGNC:34362,RNA5S1,"RNA, 5S ribosomal 1",RN5S1,,1q42.13,5S ribosomal RNAs


[1] 64

[1] 247

[1] 235

[1] 2026780

[1] 639

[1] 404

[1] 2021379


# Pairs lost due to ribosomal: 32548

In [7]:
# Polymerase genes
old = nrow(pc_hgnc)
rnapol_genes = unique(pc_hgnc$PARTICIPANT_A[grep("^RPAP|^POL", pc_hgnc$PARTICIPANT_A)])
rnapol_genes = union(rem_rib_genes, 
                      unique(pc_hgnc$PARTICIPANT_B[grep("^RPAP|^POL", pc_hgnc$PARTICIPANT_B)]))
rnapol_genes
length(rnapol_genes)
pc_hgnc <- pc_hgnc[!pc_hgnc$PARTICIPANT_A %in% rnapol_genes,]
pc_hgnc <- pc_hgnc[!pc_hgnc$PARTICIPANT_B %in% rnapol_genes,]
nrow(pc_hgnc)

cat("\n# Pairs lost due to polymerase genes:", old-nrow(pc_hgnc))

[1] "RPL12P2"        "RPL13AP2"       "RPL18AP3"       "RPL18P11"      
  [5] "RPL23AP32"      "RPL23AP5"       "RPL23AP64"      "RPL27AP6"      
  [9] "RPL29P11"       "RPL36A-HNRNPH2" "RPL37AP8"       "RPL39P5"       
 [13] "RPL4P5"         "RPL6P1"         "RPL7AP27"       "RPL7AP66"      
 [17] "RPL9P8"         "RPLP0P2"        "RPLP0P6"        "RPP14"         
 [21] "RPP21"          "RPP25L"         "RPP25"          "RPP30"         
 [25] "RPP38"          "RPP40"          "RPS10-NUDT3"    "RPS10P5"       
 [29] "RPS14P3"        "RPS15AP21"      "RPS15P2"        "RPS19BP1"      
 [33] "RPS20P14"       "RPS26P11"       "RPS26P50"       "RPS2P32"       
 [37] "RPS2P46"        "RPS2P5"         "RPS3P6"         "RPS3P7"        
 [41] "RPS4XP17"       "RPS6KA1"        "RPS6KA2"        "RPS6KA3"       
 [45] "RPS6KA4"        "RPS6KA5"        "RPS6KA6"        "RPS6KB1"       
 [49] "RPS6KB2"        "RPS6KC1"        "RPS6KL1"        "RPSA2"         
 [53] "RPSAP14"        "RPSAP15"        "RPSAP26"        "RPL12P1"       
 [57] "RPL13AP5"       "RPPH1"          "RPS2P45"        "RPL34-DT"      
 [61] "RPS7P4"         "RPS7P5"         "RPL29P30"       "RPL36AP37"     
 [65] "RPL10P16"       "RPL13P12"       "RPL21P16"       "RPL21P19"      
 [69] "RPL30P8"        "RPL7AP9"        "RPL7P32"        "RPS15AP25"     
 [73] "RPS26P54"       "RPSAP55"        "RPL26P30"       "RPL32P18"      
 [77] "RPL32P3"        "RPSAP52"        "RPL32P20"       "RPL36P14"      
 [81] "RPS12P22"       "RPS12P23"       "RPS12P4"        "RPS25P6"       
 [85] "RPS27P19"       "RPSAP19"        "RPSAP56"        "RPL21P68"      
 [89] "RPL21P87"       "RPL23AP82"      "RPL29P2"        "RPLP0P1"       
 [93] "RPS15AP10"      "RPL10P12"       "RPL21P44"       "RPL27P2"       
 [97] "RPL29P15"       "RPS2P35"        "RPL13AP20"      "RPL13AP6"      
[101] "RPL21P28"       "RPL23AP7"       "RPL23AP87"      "RPLP2P3"       
[105] "RPS26P35"       "RPS27P29"       "RPL35P5"        "RPS18P5"       
[109] "RPL21P97"       "RPL23AP37"      "RPL13AP3"       "RPL12P14"      
[113] "RPL12P19"       "RPL12P32"       "RPL12P35"       "RPS10P11"      
[117] "RPS10P13"       "RPS10P22"       "RPS10P4"        "RPS10P7"       
[121] "RPS18P12"       "RPS25P8"        "RPSAP18"        "RPSAP29"       
[125] "RPSAP61"        "RPSAP8"         "RPSAP9"         "RPL23AP42"     
[129] "RPL24P4"        "RPL13AP17"      "RPL18P4"        "RPL15P11"      
[133] "RPL17P33"       "RPL17P50"       "RPL21P62"       "RPL26P11"      
[137] "RPL7AP36"       "RPS27AP17"      "RPS2P18"        "RPS2P28"       
[141] "RPSAP30"        "RPSAP47"        "RPL7P23"        "RPL3P4"        
[145] "RPL41P5"        "RPL6P27"        "RPL7AP6"        "RPL7P1"        
[149] "RPL7P9"         "RPS13P2"        "RPS28P7"        "RPL23AP53"     
[153] "RPS4XP21"       "RPS17P16"       "RPS15P5"        "RPL26P32"      
[157] "RPL7P26"        "RPL14P1"        "RPL23AP44"      "RPS24P9"       
[161] "RPS8P10"        "RPL36AP33"      "RPL10AP2"       "RPL10P9"       
[165] "RPL17P7"        "RPS3AP47"       "RPS16P5"        "RPS26P57"      
[169] "RPL10P7"        "RPL12P27"       "RPL23AP93"      "RPL7P24"       
[173] "RPL7P50"        "RPP38-DT"       "RPS26P21"       "RPS2P36"       
[177] "RPS7P12"        "RPL23P8"        "RPL5P29"        "RPSAP12"       
[181] "RPSAP28"        "RPSAP31"        "RPSAP3"         "RPS4XP3"       
[185] "RPL10P8"        "RPLP2P1"        "RPS4XP9"        "RPS4XP2"       
[189] "RPL22P11"       "RPS18P9"        "RPL10AP3"       "RPL21P131"     
[193] "RPS17P10"       "RPS10P2"        "RPS26P31"       "RPL31P46"      
[197] "RPL12P38"       "RPL13AP25"      "RPL21P110"      "RPL21P12"      
[201] "RPL21P13"       "RPL23AP52"      "RPL26P6"        "RPL27AP"       
[205] "RPL5P30"        "RPL7P4"         "RPS12P26"       "RPS4XP16"      
[209] "RPS4XP1"        "RPL11P1"        "RPL23AP49"      "RPL32P27"      
[213] "RPL15P17"       "RPL19P9"        "RPL23P6"        "RPL7P59"       
[217] 

[1] 468

[1] 2008231


# Pairs lost due to polymerase genes: 13148

## 3. Exclude pairs within 50kb of each other and in MHC complex

### A. Get the hg38 based loci (using same input as MAGMA)

In [8]:
loci <- fread("~/Desktop/SC_GWAS_Bench/data/MAGMA/NCBI38/NCBI38.gene.loc")
loci[1:2,]

V1,V2,V3,V4,V5,V6
<int>,<chr>,<int>,<int>,<chr>,<chr>
79501,1,69091,70008,+,OR4F5
100996442,1,141934,174394,-,LOC100996442


In [9]:
# see which ones can be kept cuz MAGMA equivalent
old = nrow(pc_hgnc)
pc_hgnc_magma <- pc_hgnc[pc_hgnc$PARTICIPANT_A %in% loci$V6 & pc_hgnc$PARTICIPANT_B %in% loci$V6,]
dim(pc_hgnc_magma)

loci_partA = loci[loci$V6 %in% pc_hgnc$PARTICIPANT_A,]
loci_partA = loci_partA[order(loci_partA$V6),]
colnames(loci_partA) = c("Orig_Name", "A_Chrom", "A_Start", "A_End", "Strand", "A_Gene")

loci_partB = loci[loci$V6 %in% pc_hgnc$PARTICIPANT_B,]
loci_partB = loci_partB[order(loci_partB$V6),]
colnames(loci_partB) = c("Orig_Name", "B_Chrom", "B_Start", "B_End", "Strand", "B_Gene")
cat("\n# Pairs lost due to not being in MAGMA loci:", old-nrow(pc_hgnc_magma))

[1] 1136672       7


# Pairs lost due to not being in MAGMA loci: 871559

In [10]:
pc_hgnc_magma_new <- merge(pc_hgnc_magma, loci_partA[,c("A_Chrom", "A_Start", "A_End", "A_Gene")], by.x = "PARTICIPANT_A", by.y = "A_Gene", all.x = TRUE)
pc_hgnc_magma_new <- merge(pc_hgnc_magma_new, loci_partB[,c("B_Chrom", "B_Start", "B_End", "B_Gene")], by.x = "PARTICIPANT_B", by.y = "B_Gene", all.x = TRUE)
loci_partA[loci_partA$V6 == "Gene",]
pc_hgnc_magma_new[1:3,]

Orig_Name,A_Chrom,A_Start,A_End,Strand,A_Gene
<int>,<chr>,<int>,<int>,<chr>,<chr>


PARTICIPANT_B,PARTICIPANT_A,INTERACTION_TYPE,INTERACTION_DATA_SOURCE,INTERACTION_PUBMED_ID,PATHWAY_NAMES,MEDIATOR_IDS,A_Chrom,A_Start,A_End,B_Chrom,B_Start,B_End
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<int>
A1BG,ABCG2,controls-state-change-of,CTD,23896426;37660740,,ctdbase:chemical_mesh_d002104;ctdbase:ABU_MESH_D002104;ctdbase:B_4750700;ctdbase:RXN_8494535,4,88090264,88231417,19,58346806,58353499
A1BG,ATP7A,controls-state-change-of,CTD,11350187;14985339;15467011;15923132;21336677;23345593;23776592;23896426;24522273;25247420;32278528,,ctdbase:REC_6713796;ctdbase:ABU_3770939;ctdbase:ABU_3792340;ctdbase:UPT_6713802;ctdbase:ABU_3770941;ctdbase:chemical_mesh_d003300;ctdbase:ABU_4940882;ctdbase:ABU_MESH_D003300;ctdbase:B_4750526;ctdbase:ABU_3727198,X,77910656,78050395,19,58346806,58353499
A1BG,ATP7B,controls-state-change-of,CTD,16549536;23896426;24368744;24522273;32278528,,ctdbase:ABU_4773668;ctdbase:ABU_4772080;ctdbase:ABU_3767626;ctdbase:ABU_4773673;ctdbase:ABU_4773671;ctdbase:UPT_6713798;ctdbase:chemical_mesh_d003300;ctdbase:ABU_MESH_D003300;ctdbase:REC_6713792;ctdbase:B_4750526,13,51891086,52012099,19,58346806,58353499


In [11]:
sort(unique(pc_hgnc$INTERACTION_TYPE))
sort(unique(pc_hgnc_magma_new$INTERACTION_TYPE))

[1] ""                               "catalysis-precedes"            
 [3] "chemical-affects"               "consumption-controlled-by"     
 [5] "controls-expression-of"         "controls-phosphorylation-of"   
 [7] "controls-production-of"         "controls-state-change-of"      
 [9] "controls-transport-of"          "controls-transport-of-chemical"
[11] "interacts-with"                 "reacts-with"                   
[13] "used-to-produce"

[1] "catalysis-precedes"          "controls-expression-of"     
[3] "controls-phosphorylation-of" "controls-state-change-of"   
[5] "controls-transport-of"       "interacts-with"

### B. Remove those gene pairs overlapping and/or within 50kb of each other

In [12]:
# remove those that are within 50kb of one another
# add unique id
pc_hgnc_magma_new$unique_id = seq(1, nrow(pc_hgnc_magma_new))
# get those that should be removed
## see which have same chromosome
remove = pc_hgnc_magma_new[pc_hgnc_magma_new$A_Chrom == pc_hgnc_magma_new$B_Chrom,]
nrow(remove)
## Get the distances between them
remove$DIFF1 = abs(remove$A_Start - remove$B_Start)
remove$DIFF2 = abs(remove$A_Start - remove$B_End)
remove$DIFF3 = abs(remove$A_End - remove$B_End)
remove$DIFF4 = abs(remove$A_End - remove$B_Start)

# get those overlapping
overlap <- remove[(remove$A_Start <= remove$B_End) & (remove$B_Start <= remove$A_End), ]
dim(overlap)
## get those within 50kb
remove = remove[remove$DIFF1 < 50000 | remove$DIFF2 < 50000 | remove$DIFF3 < 50000 | remove$DIFF4 < 50000,]
dim(remove)

remove[1:3,]

remove_ids = union(overlap$unique_id, remove$unique_id)
length(remove_ids)



[1] 61649

[1] 165  18

[1] 868  18

PARTICIPANT_B,PARTICIPANT_A,INTERACTION_TYPE,INTERACTION_DATA_SOURCE,INTERACTION_PUBMED_ID,PATHWAY_NAMES,MEDIATOR_IDS,A_Chrom,A_Start,A_End,B_Chrom,B_Start,B_End,unique_id,DIFF1,DIFF2,DIFF3,DIFF4
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
ABCA10,ABCA5,catalysis-precedes,Reactome,11178988;11309290;11435397;11478798;11606068;12150964;12183068;12504089;12821155;24831815,ABC transporters in lipid homeostasis,reactome:Catalysis2;http://bioregistry.io/reactome:R-HSA-8848053;reactome:Catalysis6;http://bioregistry.io/reactome:R-HSA-1454928,17,69244435,69327182,17,69148007,69244815,1310,96428,380,82367,179175
ABCA6,ABCA10,interacts-with,BioGRID,33961781,,biogrid:MolecularInteraction_c754f6e2-156a-4282-8a92-8b5716e31917___null__816562_,17,69148007,69244815,17,69078089,69141992,1352,69918,6015,102823,166726
ABCC12,ABCC11,interacts-with,BioGRID,33961781,,biogrid:MolecularInteraction_0814948a-71c8-44f3-96a1-b7676f60558c___null__843848_,16,48166910,48235177,16,48082973,48146770,1709,83937,20140,88407,152204


[1] 870

In [13]:
# only keep those not overlapping significantly
old = nrow(pc_hgnc_magma_new)
pc_hgnc_magma_new <- pc_hgnc_magma_new[!pc_hgnc_magma_new$unique_id %in% remove_ids,]
nrow(pc_hgnc_magma_new)
cat("\n# Pairs lost due to overlap or within 50kb:", old-nrow(pc_hgnc_magma_new))

[1] 1135802


# Pairs lost due to overlap or within 50kb: 870

### C. Remove those within the MHC region

In [14]:
# remove anything within the MHC region
pc_hgnc_magma_new[1:2,]
old = nrow(pc_hgnc_magma_new)
remove_mhc = pc_hgnc_magma_new[pc_hgnc_magma_new$A_Chrom == "6" & 
                               pc_hgnc_magma_new$A_Start > 29600000 & 
                               pc_hgnc_magma_new$A_End < 33300000,]$unique_id
length(remove_mhc)
remove_mhc=c()
remove_mhc = union(remove_mhc, pc_hgnc_magma_new[pc_hgnc_magma_new$B_Chrom == "6" & 
                               pc_hgnc_magma_new$B_Start > 29600000 & 
                               pc_hgnc_magma_new$B_End < 33300000,]$unique_id)

length(remove_mhc)

pc_hgnc_magma_new = pc_hgnc_magma_new[!pc_hgnc_magma_new$unique_id %in% remove_mhc,]
nrow(pc_hgnc_magma_new)
cat("\n# Pairs lost due to being within MHC region:", old-nrow(pc_hgnc_magma_new))

PARTICIPANT_B,PARTICIPANT_A,INTERACTION_TYPE,INTERACTION_DATA_SOURCE,INTERACTION_PUBMED_ID,PATHWAY_NAMES,MEDIATOR_IDS,A_Chrom,A_Start,A_End,B_Chrom,B_Start,B_End,unique_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<int>,<int>
A1BG,ABCG2,controls-state-change-of,CTD,23896426;37660740,,ctdbase:chemical_mesh_d002104;ctdbase:ABU_MESH_D002104;ctdbase:B_4750700;ctdbase:RXN_8494535,4,88090264,88231417,19,58346806,58353499,1
A1BG,ATP7A,controls-state-change-of,CTD,11350187;14985339;15467011;15923132;21336677;23345593;23776592;23896426;24522273;25247420;32278528,,ctdbase:REC_6713796;ctdbase:ABU_3770939;ctdbase:ABU_3792340;ctdbase:UPT_6713802;ctdbase:ABU_3770941;ctdbase:chemical_mesh_d003300;ctdbase:ABU_4940882;ctdbase:ABU_MESH_D003300;ctdbase:B_4750526;ctdbase:ABU_3727198,X,77910656,78050395,19,58346806,58353499,2


[1] 10290

[1] 9513

[1] 1126289


# Pairs lost due to being within MHC region: 9513

* excluded those interactions that were annotated as in-complex-with, because those genes tended to be co-expressed and might inflate the result
* excluded 2291 ribosomal genes and housekeeping genes defined by the HSIAO_HOUSEKEEPING_GENES set from MSigDB (expressed across 19 tissues 
* excluded those pairs that were located within 50 kb of each other (since 50kb window)
* excluded pairs in the MHC complex: MHC region (chr6:26000000_34000000, hg19)

### Final Comparison to original

In [15]:
nrow(pc_hgnc_magma_new)

[1] 1126289

In [16]:
# original pathway
pathway <- fread("~/Downloads/PathwayCommons12.All.hgnc.exPCDHA.MHC.NCBI37.tsv", 
                header=FALSE)
pathway[1:2,]
dim(pathway)

V1,V2
<chr>,<chr>
A1BG,A2M
A1BG,ABCC6


[1] 805375      2

In [17]:
nrow(pc_hgnc_magma_new) - nrow(pathway)
new_genes = union(pc_hgnc_magma_new$PARTICIPANT_A, pc_hgnc_magma_new$PARTICIPANT_B)
old_genes = union(pathway$V1, pathway$V2)
length(new_genes)
length(old_genes)
length(setdiff(new_genes, old_genes))
setdiff(new_genes, old_genes)
length(setdiff(old_genes, new_genes))
setdiff(old_genes, new_genes)[1:5]

[1] 320914

[1] 17041

[1] 18246

[1] 566

[1] "HSPA1B"         "MT1A"           "NOG"            "UGT1A3"        
  [5] "ACOT1"          "ACOT4"          "ACY3"           "NME1"          
  [9] "NDUFS8"         "NME1-NME2"      "TPSAB1"         "CYP11B1"       
 [13] "IGF2"           "NPPB"           "CYP1A2"         "EPO"           
 [17] "CGB5"           "CGB8"           "GSTM2"          "HOXA9"         
 [21] "GRXCR2"         "MPP1"           "OTOG"           "GNG11"         
 [25] "FGF4"           "AOC2"           "AOC3"           "ADH1A"         
 [29] "ADH1C"          "LIPA"           "SLC18A3"        "FLOT1"         
 [33] "CHRNB4"         "CNGA4"          "AP2M1"          "PSMD3"         
 [37] "ADAM18"         "APOA5"          "APOC3"          "CLCN2"         
 [41] "HLA-DRB1"       "MPC1L"          "HSPA1L"         "PSME2"         
 [45] "PABPN1L"        "IL13"           "MAP4K2"         "MPDU1"         
 [49] "CCL11"          "CCL7"           "CXCL9"          "CXCR1"         
 [53] "PTGIR"          "ALOX15B"        "TST"            "ALOXE3"        
 [57] "CYP27B1"        "AMY1A"          "ANKRD30A"       "APOA4"         
 [61] "APOC1"          "APOC2"          "AIFM3"          "APOBEC3G"      
 [65] "PA2G4"          "ARF5"           "ALYREF"         "ATP4A"         
 [69] "SULT1A2"        "AKR7A3"         "OTOP1"          "APOBEC3D"      
 [73] "HAUS4"          "BAG6"           "ANKLE1"         "MMP10"         
 [77] "FGB"            "FGG"            "AZU1"           "LTA"           
 [81] "LTB"            "CBX4"           "CIAO1"          "UXT"           
 [85] "C1QA"           "C1QB"           "BAGE2"          "CXCL11"        
 [89] "C5AR1"          "CARD16"         "MT3"            "TH"            
 [93] "C10orf120"      "C3orf18"        "CCR2"           "CDKN2B"        
 [97] "CCL3"           "CCL8"           "ACTRT2"         "ANKRD16"       
[101] "HLA-DPB1"       "CD1C"           "ST6GALNAC6"     "CCT8L2"        
[105] "BIVM-ERCC5"     "C2CD4B"         "CDHR4"          "C2CD4A"        
[109] "CD7"            "C8orf58"        "CENPH"          "CENPB"         
[113] "C22orf42"       "ACRV1"          "FUT5"           "ARL9"          
[117] "GSTM5"          "CLEC4M"         "ARMS2"          "LIN7B"         
[121] "CYP26C1"        "CASP5"          "PTPMT1"         "ANKRD36C"      
[125] "C1orf146"       "CXCL3"          "CXCL6"          "A3GALT2"       
[129] "CYP2A7"         "CYP4F3"         "C1orf53"        "CRYBB1"        
[133] "COL11A2"        "ADM5"           "ANKRD34B"       "C11orf96"      
[137] "ECH1"           "CHIC1"          "DEF8"           "DNAJC25"       
[141] "DHRS12"         "CEACAM16"       "DPPA2"          "CT45A6"        
[145] "DHRS4L1"        "DUOXA2"         "CDCP2"          "DPEP3"         
[149] "EID2B"          "DEFB106A"       "DEFB106B"       "CRYGD"         
[153] "DEFB135"        "DEFB136"        "TAP1"           "TAP2"          
[157] "ZNF830"         "F10"            "DEFB125"        "ERVMER34-1"    
[161] "FCN2"           "SRMS"           "GADL1"          "FOLR2"         
[165] "MST1"           "DEFB110"        "FNDC1"          "GABARAP"       
[169] "FCRL6"          "C7orf57"        "GBX1"           "HOXA11"        
[173] "GAGE12J"        "GAGE2A"         "FAM25A"         "FAM25G"        
[177] "GJB6"           "GOLGA6C"        "CSH1"           "CSH2"          
[181] "HBG2"           "HBG1"           "HBQ1"           "HLA-C"         
[185] "FBXO47"         "HEXIM2"         "BPIFC"          "DEFB107A"      
[189] "DEFB107B"       "HSPB2"          "HSPE1"          "HLA-DMB"       
[193] "DTHD1"          "IFNA4"          "IFNA6"          "IFNA8"         
[197] "IFNL2"          "KLK2"           "HSPE1-MOB4"     "IFITM5"        
[201] "IFIT5"          "IFIT2"          "HOXB7"          "TLR10"         
[205] "HBM"            "CRYGB"          "STATH"          "FHAD1"         
[209] "HHLA1"          "DEFA3"          "KRT17"          "KRT16"         
[213] "KRT19"          "KRT2"           "KRT14"          "KRT6A"         
[217] 

[1] 1771

[1] "AAMP"     "ABCA11P"  "ABHD18"   "ABLIM1"   "ABRAXAS1"

In [18]:
rem_hk_genes = c("HSPA1B", "PSMD3", "ACTRT2", "TST", "HSPA1L", "PABPN1L")
old = nrow(pc_hgnc_magma_new)
pc_hgnc_magma_new = pc_hgnc_magma_new[!pc_hgnc_magma_new$PARTICIPANT_A %in% rem_hk_genes,]
pc_hgnc_magma_new = pc_hgnc_magma_new[!pc_hgnc_magma_new$PARTICIPANT_B %in% rem_hk_genes,]
cat("\n# Pairs lost due to added hand annotated housekeeping genes:", old-nrow(pc_hgnc_magma_new))
nrow(pc_hgnc_magma_new)


# Pairs lost due to added hand annotated housekeeping genes: 1872

[1] 1124417

In [19]:
# Save file in the same format as the original
write.table(pc_hgnc_magma_new[,c("PARTICIPANT_A", "PARTICIPANT_B")], 
            "~/Desktop/SCRNA-GWAS-Benchmarking/data/PathwayCommons14.All.hgnc.exPCDHA.MHC.NCBI38.tsv", 
            row.names=FALSE, col.names=FALSE, quote=FALSE, sep="\t")